##  Sentinel-3 SLSTR tiling

### Service Definition

In [1]:
service = dict([('title', 'Sentinel-3 SLSTR Level-2 reprojection and tiling'),
                ('abstract', 'This service takes as input a Sentinel-3 SLSTR Level 2 (SL_2_LST____) product on DESCENDING pass and does the reprojection and tiling'),
                ('identifier', 'ewf-wfp-03-01-02')])

### Parameter Definition 

### Runtime parameter definition

**Input reference**

The input identifier is the catalogue entry URL (a.k.a. self value).

In [2]:
input_reference = dict([('identifier', 'input_reference'),
                        ('title', 'Sentinel-3 SLSTR Level-2 (SL_2_LST____ descending pass)'),
                        ('abstract', 'This service takes as input a Sentinel-3 SLSTR Level 2 (SL_2_LST____) product on DESCENDING pass'),
                        ('value', 'https://catalog.terradue.com/sentinel3/search?format=json&uid=S3A_SL_2_LST____20190107T082459_20190107T100558_20190108T183657_6059_040_064______LN2_O_NT_003'),
                        ('stac:collection', 'input_reference'),
                        ('stac:href', 'catalog.json'),
                        ('max_occurs', '16')])

In [3]:
tiling_level = dict([('identifier', 'tiling_level'),
                ('value', '4'),
                ('title', 'Tiling level'),
                ('abstract', 'Tiling level'),
                ('max_occurs', '1')])


In [4]:
aoi = dict([('identifier', 'aoi'),
                ('value', 'POLYGON((21.5 18.39, 37.82 18.39, 37.82 -2.09, 21.5 -2.09, 21.5 18.39))'),
                ('title', 'Area of Interest'),
                ('abstract', 'Area of Interest'),
                ('max_occurs', '1')])


**Data path**

This path defines where the data is staged-in. 

In [5]:
data_path = '/workspace/data/s3'

In [6]:
input_catalog = '/workspace/data/s3/catalog.json'

### Workflow

#### Import the packages

In [7]:
import os
import sys
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
#os.environ['_JAVA_OPTIONS'] = '-Xms41g -Xmx41g'

sys.path.append('.')
import gdal
from helpers import *
from shapely.wkt import loads
from shapely.geometry import box
from shapely.geometry import shape
import shutil
from pystac import Catalog, Collection, Item, MediaType, Asset, CatalogType

from tiling import s3_tiles
import time
import glob
import numpy as np
gdal.UseExceptions()

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
cat = Catalog.from_file(input_catalog)

if cat is None:
    raise ValueError()

In [10]:
collection = next(cat.get_children())

In [11]:
item = next(collection.get_items())

In [12]:
item.properties['eop:orbitDirection']

'DESCENDING'

In [13]:
if item.properties['eop:orbitDirection'] != 'DESCENDING':
    ciop.log('ERROR','Product cannot be used as input')
    raise Exception('Use products with descending orbit direction')

In [14]:
s3_wkt = shape(item.geometry).wkt

In [15]:
s3_wkt

'POLYGON ((-104.938 -85.05115000000001, -68.2803 -85.05115000000001, -37.8937 -84.61020000000001, -21.3075 -82.5334, -12.1643 -80.1264, -6.53162 -77.5792, -2.70693 -74.9623, 2.51071153478015e-18 -72.3920772829885, 0.090445 -72.3062, 2.25453 -69.62569999999999, 4.00362 -66.92870000000001, 5.46717 -64.21980000000001, 6.72631 -61.502, 7.83487 -58.777, 8.82954 -56.0462, 9.736330000000001 -53.3102, 10.5743 -50.5699, 11.3576 -47.8255, 12.097 -45.0775, 12.8012 -42.3262, 13.4766 -39.5718, 14.129 -36.8144, 14.7626 -34.0544, 15.3813 -31.2918, 15.9882 -28.5269, 16.5862 -25.7599, 17.1776 -22.991, 17.7648 -20.2204, 18.3496 -17.4483, 18.934 -14.6749, 19.5198 -11.9005, 20.1087 -9.12541, 20.7025 -6.34985, 21.3029 -3.57415, 21.9116 -0.798609, 22.5306 1.97642, 23.1619 4.75059, 23.8075 7.52352, 24.4699 10.2948, 25.1515 13.064, 25.8552 15.8306, 26.5843 18.5942, 27.3423 21.3541, 28.1334 24.1098, 28.9626 26.8606, 29.8353 29.6058, 30.7582 32.3443, 31.7391 35.0753, 32.7873 37.7975, 33.9141 40.5095, 35.1332 43

In [16]:
aoi_wkt = loads(aoi['value']).wkt

In [17]:
aoi_wkt

'POLYGON ((21.5 18.39, 37.82 18.39, 37.82 -2.09, 21.5 -2.09, 21.5 18.39))'

### Import Sentinel-3 SLSTR product

In [18]:
operators = ['Read',
             'Subset',
             'Reproject',
             'Write']

In [19]:
read = dict()

s3_path = item.assets['metadata'].get_absolute_href()

read['file'] =  s3_path
read['formatName'] = 'Sen3'

subset = dict()
subset['geoRegion'] = aoi_wkt
subset['copyMetadata'] = 'true'
subset['fullSwath'] = 'true'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

write = dict()
write['file'] = 's3_slstr'

In [20]:
snap_graph(os.environ['GPT_BIN'],
           operators,
           Subset=subset,
           Read=read, 
           Reproject=reproject,
           Write=write)

0

In [21]:
def s3_to_tile(input_tif, item, tile):
    
    translate_options = gdal.TranslateOptions(gdal.ParseCommandLine("-co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW"))
        
    x_min, y_min, x_max, y_max = tile.tile.bounds

    output_tile_name = '{}_L{}_C{}_R{}'.format(item.id,
                                                    tile.level,
                                                    tile.col,
                                                    tile.row)


    gdal.Translate('tmp_{}.tif'.format(output_tile_name),
                   input_tif,
                   projWin=[x_min, y_max, x_max, y_min],
                   projWinSRS='EPSG:4326',
                   xRes=0.008714267406038, 
                   yRes=0.008714267406038,
                   resampleAlg='bilinear')

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name),
                   gdal.OF_READONLY)

    gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
    ds.BuildOverviews('NEAREST', [2,4,8,16,32])
    ds = None

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name))
    ds = gdal.Translate('{}.tif'.format(output_tile_name), ds, options=translate_options)
    ds = None

    band_names = ['LST', 'NDVI', 'Land mask', 'Cloud mask']

    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)

    
    for index in range(ds.RasterCount):

        srcband = ds.GetRasterBand(index+1)

        srcband.SetDescription(band_names[index])

        # if NDVI band set noData value as 255
        if index == 1:
            
            srcband.SetNoDataValue(255)
    
    ds.FlushCache()

    ds = None
    ###Drop generated tiles where land is less than Threshold=0.05
    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)
    land_mask = np.array(ds.GetRasterBand(3).ReadAsArray(), dtype= np.uint8)
    
    
    if np.count_nonzero(land_mask)/land_mask.size < 0.05 : 
        print('removed {}.tif\n'.format(output_tile_name))
        print('land percentage {}\n ********\n '.format(np.count_nonzero(land_mask)/land_mask.size))
        os.remove('{}.tif'.format(output_tile_name))
    else: 
        ndvi_tmp=ds.GetRasterBand(2).ReadAsArray()
        cloud_tmp = np.array(ds.GetRasterBand(4).ReadAsArray(), dtype= np.uint8)
        # If land_mask=0 set data to noData
    
        masked_data = lambda x,y: 255 if y==0  or x>1 or x<-1 else x
        vfunc_masked = np.vectorize(masked_data, otypes=[np.float])
    
        updated_ndvi_data = vfunc_masked(ndvi_tmp, land_mask)
        if updated_ndvi_data.max()==updated_ndvi_data.min():
            os.remove('{}.tif'.format(output_tile_name))
            print('removed {}.tif\n'.format(output_tile_name))
            print('land percentage {} OK but ndvi min {} and ndvi max {}\n ********\n '.format(np.count_nonzero(land_mask)/land_mask.size,updated_ndvi_data.min(),updated_ndvi_data.max()))
        else:
    
            ds.GetRasterBand(2).WriteArray(updated_ndvi_data)
            #Render the land & cloud masks boolean on borders
            ds.GetRasterBand(3).WriteArray(land_mask)
            ds.GetRasterBand(4).WriteArray(cloud_tmp)
            with open(output_tile_name + '.properties', 'w') as file:

                file.write('title=Tile L:{1} C:{2} R:{3} {0}\n'.format(item.id,
                                                                  tile.level, 
                                                                  tile.col, 
                                                                  tile.row))

                date='{}/{}'.format(item.datetime.strftime('%Y-%m-%dT%H:%M:%SZ'), 
                                             item.datetime.strftime('%Y-%m-%dT%H:%M:%SZ'))

                file.write('date={}\n'.format(date))

                file.write('geometry={0}'.format(tile.tile.wkt))


    for f in ['tmp_{}.tif'.format(output_tile_name), 'tmp_{}.tif.ovr'.format(output_tile_name)]:

        if os.path.exists(f):

            os.remove(f)
    
    return True
    

### Tiling

In [22]:
bands = [os.path.join('s3_slstr.data', '{}.img'.format(band)) for band in ['LST', 'NDVI', 'cloud_in', 'confidence_in']]
        
s3_data = read_s3(bands)
        

In [23]:
bands

['s3_slstr.data/LST.img',
 's3_slstr.data/NDVI.img',
 's3_slstr.data/cloud_in.img',
 's3_slstr.data/confidence_in.img']

In [24]:
ds = gdal.Open(bands[0])

geo_transform = ds.GetGeoTransform()
projection_ref = ds.GetProjectionRef()
    

In [25]:
lst = s3_data[:,:,0]
ndvi = s3_data[:,:,1]
cloud = s3_data[:,:,2]
confidence = s3_data[:,:,3]

In [26]:
ndvi

array([[-32768., -32768., -32768., ..., -32768., -32768., -32768.],
       [-32768., -32768., -32768., ..., -32768., -32768., -32768.],
       [-32768., -32768., -32768., ..., -32768., -32768., -32768.],
       ...,
       [-32768., -32768., -32768., ..., -32768., -32768., -32768.],
       [-32768., -32768., -32768., ..., -32768., -32768., -32768.],
       [-32768., -32768., -32768., ..., -32768., -32768., -32768.]],
      dtype=float32)

In [27]:
mask = get_slstr_confidence_mask('land', confidence)

In [28]:
cloud_mask =  get_slstr_mask('gross_cloud', cloud)

In [29]:
range_trimming = lambda x,y,z: 255 if y==0 or z==1 or x>1 or x<-1 else x
vfunc_range_trimming = np.vectorize(range_trimming,otypes=[np.float])
    
ndvi_filtered=vfunc_range_trimming(ndvi,mask,cloud_mask)

In [30]:
output_name = 'temp_sls_trimed.tif'

In [31]:
driver = gdal.GetDriverByName('GTiff')

output = driver.Create(output_name, 
                       lst.shape[1], 
                       lst.shape[0], 
                       4, 
                       gdal.GDT_Float32)

output.SetGeoTransform(geo_transform)
output.SetProjection(projection_ref)
output.GetRasterBand(1).WriteArray(lst)
output.GetRasterBand(2).WriteArray(ndvi_filtered)
output.GetRasterBand(3).WriteArray(mask)
output.GetRasterBand(4).WriteArray(cloud_mask)

output.FlushCache()

output = None

In [32]:

tiles = s3_tiles(loads(aoi['value']), int(tiling_level['value']))

In [33]:
tiles

,col,row,level,s3_tile,tile
0,18,7,4,"POLYGON ((33.75000 -2.09000, 22.50000 -2.09000...","POLYGON ((33.75000 -11.25000, 33.75000 0.00000..."
1,19,7,4,"POLYGON ((37.82000 0.00000, 37.82000 -2.09000,...","POLYGON ((45.00000 -11.25000, 45.00000 0.00000..."
2,17,7,4,"POLYGON ((22.50000 -2.09000, 21.50000 -2.09000...","POLYGON ((22.50000 -11.25000, 22.50000 0.00000..."
3,18,8,4,"POLYGON ((33.75000 0.00000, 22.50000 0.00000, ...","POLYGON ((33.75000 0.00000, 33.75000 11.25000,..."
4,17,8,4,"POLYGON ((21.50000 0.00000, 21.50000 11.25000,...","POLYGON ((22.50000 0.00000, 22.50000 11.25000,..."
5,19,8,4,"POLYGON ((37.82000 11.25000, 37.82000 0.00000,...","POLYGON ((45.00000 0.00000, 45.00000 11.25000,..."
6,19,9,4,"POLYGON ((33.75000 18.39000, 37.82000 18.39000...","POLYGON ((45.00000 11.25000, 45.00000 22.50000..."
7,18,9,4,"POLYGON ((22.50000 18.39000, 33.75000 18.39000...","POLYGON ((33.75000 11.25000, 33.75000 22.50000..."
8,17,9,4,"POLYGON ((21.50000 18.39000, 22.50000 18.39000...","POLYGON ((22.50000 11.25000, 22.50000 22.50000..."


In [34]:
for index, tile in tiles.iterrows():
    
    logging.info('Tile L{} C{} R{}'.format(tile.level,
                                       tile.col,
                                       tile.row))


    s3_to_tile(output_name, item, tile)

removed S3A_SL_2_LST____20190107T082459_20190107T100558_20190108T183657_6059_040_064______LN2_O_NT_003_L4_C18_R7.tif

land percentage 0.0
 ********
 
removed S3A_SL_2_LST____20190107T082459_20190107T100558_20190108T183657_6059_040_064______LN2_O_NT_003_L4_C19_R7.tif

land percentage 0.0
 ********
 
removed S3A_SL_2_LST____20190107T082459_20190107T100558_20190108T183657_6059_040_064______LN2_O_NT_003_L4_C17_R7.tif

land percentage 0.0
 ********
 
removed S3A_SL_2_LST____20190107T082459_20190107T100558_20190108T183657_6059_040_064______LN2_O_NT_003_L4_C19_R8.tif

land percentage 0.0
 ********
 
removed S3A_SL_2_LST____20190107T082459_20190107T100558_20190108T183657_6059_040_064______LN2_O_NT_003_L4_C19_R9.tif

land percentage 0.0
 ********
 


In [35]:
logging.info('Clean-up') 
os.remove(output_name)

shutil.rmtree('s3_slstr.data')
os.remove('s3_slstr.dim')

time.sleep(45)

for f in glob.glob('./*.tif.aux.xml'):

    os.remove(f)


### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.

opensearch-client -a better-wfp-03-01-02:AKCp5ekcV3gx7bZTjNLsoYF8Gbm2zYCGmEMQ4JcZVSfTPHeSXBt9Zf9Sky8Z1moLCQXAJ4kN7  -p count=unlimited https://catalog.terradue.com/better-wfp-03-01-02/search identifier

for uid in  $(opensearch-client -a better-wfp-03-01-02:AKCp5ekcV3gx7bZTjNLsoYF8Gbm2zYCGmEMQ4JcZVSfTPHeSXBt9Zf9Sky8Z1moLCQXAJ4kN7 -p count=unlimited https://catalog.terradue.com/better-wfp-03-01-02/search identifier); do curl -u better-wfp-03-01-02:AKCp5ekcV3gx7bZTjNLsoYF8Gbm2zYCGmEMQ4JcZVSfTPHeSXBt9Zf9Sky8Z1moLCQXAJ4kN7 -X DELETE "https://catalog.terradue.com/better-wfp-03-01-02/query?uid=${uid}";done 